In [8]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import math
import numpy.random as nrand
import numpy as np

In [3]:
stocks_prices = pd.read_csv('../data/DEFINITIVE/prices_from2008_csv.csv', parse_dates =["Date"], index_col="Date")
stocks_mc = pd.read_csv('../data/DEFINITIVE/MC_from2008_csv.csv', parse_dates =["Date"], index_col="Date")
stocks_pb = pd.read_csv('../data/DEFINITIVE/PTB_from2008_csv.csv', parse_dates =["Date"], index_col="Date")
stocks_s_c = pd.read_csv('../data/DEFINITIVE/categories_csv.csv', index_col="sector_country")
returns = stocks_prices.pct_change().dropna()
returns
# returns.iloc[0:60]
# print(returns.head())
# var = returns.iloc[0:60].var()
# std = returns.iloc[0:60].std()
# cov_matrix = returns.iloc[0:60].cov()
# print(var)
# print(std)
# cov_matrix
# # print(type(cov_matrix))
# corr_matrix = returns.iloc[0:60].corr()
# e_r = stocks_prices.iloc[0:60].resample('Y').last().pct_change().mean()
# sd = stocks_prices.iloc[0:60].pct_change().apply(lambda x: np.log(1+x)).std().apply(lambda x: x*np.sqrt(12))

,AT0000606306,AT0000652011,AT0000730007,AT0000743059,AT0000937503,BE0003470755,BE0003565737,BE0003739530,BE0003764785,BE0003797140,...,SE0000667891,SE0000667925,SE0000695876,SE0001662230,SE0005190238,SE0007100581,SE0007100599,SE0011166610,SE0012455673,SE0013256682
Date,,,,,,,,,,,,,,,,,,,,,
2008-01-31,-0.183880,-0.253608,-0.203619,-0.133887,-0.171284,-0.122362,-0.116528,0.047228,-0.018507,-0.117332,...,-0.184205,-0.076850,-0.072536,-0.161022,-0.004594,-0.154478,-0.144787,-0.067505,-0.297265,-0.144031
2008-02-29,0.010053,0.063536,0.131475,-0.002083,0.020010,-0.008811,-0.019414,-0.034631,0.032086,0.025142,...,0.172743,-0.107659,0.002595,0.062480,-0.145357,0.027653,-0.010921,0.087657,0.254511,-0.024028
2008-03-31,0.011124,0.066234,-0.069076,-0.126096,0.052632,-0.029189,-0.014399,-0.298295,-0.030499,-0.027921,...,-0.024428,-0.037473,0.072596,0.053832,0.023241,-0.038850,-0.006256,0.044358,-0.111978,-0.066052
2008-04-30,0.205559,0.161998,0.099223,0.157430,0.119318,0.168275,0.060872,0.263804,-0.003192,0.056023,...,-0.100364,0.129343,0.098954,-0.131823,0.198826,-0.126255,-0.030981,-0.041582,0.038749,0.135150
2008-05-30,-0.074448,0.028721,0.137886,0.092260,0.087716,-0.024041,-0.089571,-0.068141,0.073639,0.049620,...,0.116511,0.018673,0.025919,-0.003117,-0.037107,0.112279,-0.004670,0.098514,0.038296,0.001202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-08-31,-0.141255,-0.072781,0.051113,-0.056452,-0.098670,-0.022175,-0.075346,0.072148,-0.015385,-0.005282,...,-0.037693,-0.072665,-0.019279,0.010107,-0.076003,0.033958,-0.013533,0.005314,-0.116306,-0.043462
2018-09-28,0.012245,0.044062,-0.014706,0.060486,0.019669,0.007414,0.047386,-0.017268,-0.024089,-0.001106,...,0.014332,0.035718,0.011906,0.075969,-0.023286,-0.013667,0.041915,0.010747,0.065661,-0.076864
2018-10-31,-0.028226,0.006149,-0.089353,0.015292,-0.203300,-0.128571,-0.049922,-0.041602,-0.072048,-0.089923,...,-0.083789,0.008213,-0.032752,-0.092961,-0.031759,0.018958,-0.115116,-0.117781,-0.158454,-0.059129


# Process to follow
1.  I calculate the volatility of the returns.

2.  I classify stocks within dimensions and categories.

3.    For each category, I calculate the sum of (stock volatility/ number of stocks in the category). And then I use this number as Pij.

For example, in dimension Sectors and category Energy, there are 11 stocks. Then I calculate ∑(stock volatility / 11) = Pij

In [4]:
#calculate vol stocks in a certain period.
volatility =  returns.rolling(window=60).std() * np.sqrt(12)
stock_vol = volatility.dropna()

In [5]:
stocks_s_c = pd.read_csv('../data/DEFINITIVE/categories_csv.csv', index_col="sector_country")
#categories
#this is a list containing different sectors and countries
#I want to divide the list in different sectors
#I want to divide the list in different countries
#stocks = stocks_s_c.columns
stocks_s_c = stocks_s_c.transpose()
#stocks_s_c[stocks_s_c["Country "]== "Austria"]

# Create dictionary of stocks by country
countries = stocks_s_c["Country "].unique()
stocks_by_country = {}
for country in countries:
    stocks_by_country[country] = stocks_s_c[stocks_s_c["Country "]== country].index.values.tolist()

# Create dictionary of stocks by sector
sectors = stocks_s_c["Sector name "].unique()
stocks_by_sector = {}
for sector in sectors:
    stocks_by_sector[sector] = stocks_s_c[stocks_s_c["Sector name "]== sector].index.values.tolist()

In [6]:
stocks_by_sector

{'Financials': ['AT0000606306',
  'AT0000652011',
  'BE0003565737',
  'BE0003764785',
  'BE0003797140',
  'BE0974264930',
  'CH0011075394',
  'CH0012138530',
  'CH0012410517',
  'CH0014852781',
  'CH0021783391',
  'DE0005140008',
  'DE0008402215',
  'DE0008404005',
  'DE0008430026',
  'DE000CBK1001',
  'DK0010274414',
  'DK0010307958',
  'DK0010311471',
  'ES0113211835',
  'ES0113860A34',
  'ES0113900J37',
  'ES0115056139',
  'ES0124244E34',
  'FI0009003305',
  'FI4000297767',
  'FR0000045072',
  'FR0000120222',
  'FR0000120628',
  'FR0000120685',
  'FR0000121121',
  'FR0000121204',
  'FR0000130809',
  'FR0000131104',
  'FR0010411983',
  'GB0002162385',
  'GB0002405495',
  'GB0004082847',
  'GB0005405286',
  'GB0005603997',
  'GB0007099541',
  'GB0007668071',
  'GB0008706128',
  'GB0031348658',
  'GB00B02J6398',
  'GB00B06QFB75',
  'GB00B17BBQ50',
  'GB00B1YW4409',
  'GB00B7T77214',
  'GB00BF8Q6K64',
  'GB00BKKMKR23',
  'IT0000062072',
  'IT0000062957',
  'IT0000072618',
  'IT000348702

In [10]:
# numpy.datetime64
# I want to divide stocks in quintiles of mc for the year 2008
# I want to divide stocks in quintiles of pb for the year 2008
stocks_mc
stocks_pb
# quintiles = pd.quantile(q=0.2, axis='columns', numeric_only=True, interpolation='linear')

dates =  pd.DatetimeIndex(stocks_mc.index)
years = dates.year.unique()
months = dates.month.unique()
print(months)

In [ ]:

# stocks_mc_year = stocks_mc[ stocks_mc.index ]
# print(stocks_mc[ stocks_mc.index.year == 2008])
stocks_names = stocks_mc.columns.values.tolist()
mean_by_year = {}
for year in years:
    mean_by_year[year] = stocks_mc[ stocks_mc.index.year == year].mean()

mean_by_year = pd.DataFrame.from_dict(mean_by_year)
df_2008 = mean_by_year[2008]
# printdf_2008
# = df_2008.quantile(q=0.2)
quantile_2008 = pd.qcut(df_2008, 5, labels=range(5))

df_mc_quint = {}
for year in years:
    df_mc_quint[year] = pd.qcut(mean_by_year[year], 5, labels=range(5))

df_mc_quint = pd.DataFrame.from_dict(df_mc_quint)




mean_by_year = {}
for year in years:
    mean_by_year[year] = stocks_pb[ stocks_pb.index.year == year].mean()

mean_by_year = pd.DataFrame.from_dict(mean_by_year)

df_pb_quint = {}
for year in years:
    df_pb_quint[year] = pd.qcut(mean_by_year[year], 5, labels=range(5))

df_pb_quint = pd.DataFrame.from_dict(df_pb_quint)

In [ ]:
df_pb_quint

In [ ]:

# def entropy_calculation ( dimension, vol_stock, year):
#     for category in dimension:
#         for stock in category:
#             weighted_vol_stock = sum(vol_stock/len(category))
#
#             pij = sum(weighted_vol_stock)
#
#         vol_category_full = sum(pij)*np.log(sum(pij))
#
#     for category in dimension:
#         vol_dimension = -sum(vol_category_full)
#
#     return vol_dimensi



In [ ]:
import time
dates  = pd.DatetimeIndex(stock_vol.index)
years = dates.year.unique()
volatility_by_year = {}
# print(years)
for year in years:
     volatility_by_year[year] = stock_vol[ stock_vol.index.year == year].mean()
        
#


# print(weights)
num_portfolios = 2
counter = 0
start = time.time()
porfolios_entropy = {}
for portfolio in range(num_portfolios):
    num_stocks = len(stocks_s_c)
    weights = np.random.random(num_stocks)
    weights = weights/np.sum(weights)
    stock_weights ={}
    for i, stock in enumerate(stocks_names):
        stock_weights[stock] = weights[i]


    sectors = stocks_by_sector.keys()
    dimension = [sectors]
    dimension_sector = {}
    for year in years:
        pij = 0
        for sector in stocks_by_sector:
            stocks = stocks_by_sector[sector]
            for stock in stocks:
                temp = volatility_by_year[year]
                volatility = ((temp[temp.index == stock].iloc[0]) * stock_weights[stock])/len(stocks)
                # print(volatility)


    # * weight[stock]
                pij += volatility
                # break
                # print(volatility)
            # break
            # print(pij,sector)
        # print(pij,year)
        pij = pij * np.log(pij)
        dimension_sector[year] = pij

        # break
        # print(pij)
        # print(pij,year)
        # print()
    porfolios_entropy[portfolio] = dimension_sector
print("Total time = " +str(float(time.time()-start))+ " seconds")

In [ ]:
#     dimension_country = {}
#     for year in years:
#         pij = 0
#         for country in stocks_by_country:
#             stocks = stocks_by_country[country]

#             for stock in stocks:
#                 for weight in weights: 
#                     temp = volatility_by_year[year]
# #                     print(stock)
#                     volatility = (temp[temp.index == stock].iloc[0]* weight)/len(stocks) # * weight[stock]
#                     pij += volatility
#                 # print(volatility)
#             # print(pij,sector)
#         # print(pij,year)
#         pij = pij * np.log(pij)
#         dimension_country[year] = pij



#     dimension_mc = {}
#     for year in years:
#         pij = 0
#         for quintil in range(5):
#             stocks = df_mc_quint[year][df_mc_quint[year]==quintil].index
#     #         print(stocks)
#             for stock in stocks:
#                 for weight in weights: 
#                     temp = volatility_by_year[year]
#                     # print(stock)
#                     volatility = (temp[temp.index == stock].iloc[0]* weight)/len(stocks) # * weight[stock]
#                     pij += volatility
#                 # print(volatility)
#             # print(pij,sector)
#         # print(pij,year)
#         pij = pij * np.log(pij)
#         dimension_mc[year] = pij

#     dimension_pb = {}
#     for year in years:
#         pij = 0
#         for quintil in range(5):
#             stocks = df_pb_quint[year][df_pb_quint[year]==quintil].index
#             # print(stocks)
#             for stock in stocks:
#                 for weight in weights: 
#                     temp = volatility_by_year[year]
#                     # print(stock)
#                     volatility = (temp[temp.index == stock].iloc[0]* weight[stock])/len(stocks) # * weight[stock]
#                     pij += volatility
#                     # print(volatility)
#             # print(pij,sector)
#         # print(pij,year)
#         pij = pij * np.log(pij)
#         dimension_pb[year] = pij
        
#     print(counter)
#     counter+=1


#     print(dimension_sector)
#     print(dimension_country)
#     print(dimension_mc)
#     print(dimension_pb)
    
# end = time.time()
# print(end - start) 
    
# portfolio_entropy = {}
# for year in years:
#     sum_total = 0
#     sum_total+= dimension_sector[year]+ dimension_country[year]
#     sum_total+= dimension_mc[year]+dimension_pb[year]
#     sum_total = sum_total * -1
#     portfolio_entropy[year] = sum_total
# print(portfolio_entropy)


### 

In [ ]:
portfolio_entropy = {}
for year in years:
    sum_total = 0
    sum_total+= dimension_sector[year]+ dimension_country[year]
    sum_total+= dimension_mc[year]+dimension_pb[year]
    sum_total = sum_total * -1
    portfolio_entropy[year] = sum_total
print(portfolio_entropy)


#maximize the function using argmax



In [ ]:
# weight = np.array(range(len(stocks_s_c)))
# # print(len(stocks_s_c))
# print(weights)
num_stocks = len(stocks_s_c)
weight = np.random.random(num_stocks)
weight = weights/np.sum(weights)

    
weight[33]

In [ ]:
optimization = np.argmax(portfolio_entropy, axis = 0)
print(optimization)

In [ ]:
p_ret = []
p_vol = []
p_weights = []
p_sharpe=[]

num_stocks = len(stocks.columns)
num_portfolios = 10


In [ ]:
rf = (3.491/100)

for portfolio in range(num_portfolios):
    weights = np.random.random(num_stocks)
    weights = weights/np.sum(weights)
    p_weights.append(weights)
    returns_p = np.dot(weights, e_r)
    p_ret.append(returns_p)
    var = cov_matrix.mul(weights, axis=0).mul(weights, axis=1).sum().sum()
    sd = np.sqrt(var)
    ann_sd = sd*np.sqrt(250)
    p_vol.append(ann_sd)
    
    
    
    sharpe_p = ((p_ret[-1]-rf)/p_vol[-1])
#     print(sharpe_ratio)
    p_sharpe.append(sharpe_p)
    

In [ ]:
i=0
j=60
num_stocks = len(stocks.columns)
num_portfolios = 10000

while j<132-12:
    